In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

## Loading Environment File

In [3]:
# Try loading from environment first, then from dev.env file
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    secret_path = Path("../.env")
    print("Looking for dev.env at:", secret_path.resolve())
    if secret_path.exists():
        load_dotenv(secret_path)
        GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in environment variables or dev.env file.")

print("GOOGLE_API_KEY loaded:", bool(GOOGLE_API_KEY))

GOOGLE_API_KEY loaded: True


## Setup: Imports, Retry Config

In [1]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agents Creation

In [8]:
# RAG system
rag_agent = Agent(
    name="RAG_Vaccine_Informer",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    instruction="""XXX""",
    tools=[],
    output_key="rag_output",  # The result of this agent will be stored in the session state with this key.
)

print("✅ RAG created.")

✅ RAG created.


In [9]:
# Health Researcher: Focuses on medical breakthroughs.
sentiment_agent = Agent(
    name="sentiment_analysis",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""XXX""",
    tools=[],
    output_key="sentiment_output",  # The result will be stored with this key.
)

print("✅ sentiment_agent created.")

✅ sentiment_agent created.


In [10]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these findings into a single executive summary:

    **RAG output:**
    {rag_output}
    
    **User sentiment analysis:**
    {sentiment_output}
""",
    output_key="final_output",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [11]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_rag_sentiment_agent = ParallelAgent(
    name="ParallelRAGAndSentimentTeam",
    sub_agents=[sentiment_agent, rag_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_rag_sentiment_agent, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


## Engine Def

In [12]:
runner = InMemoryRunner(agent=root_agent)

## Testing

In [ ]:
response = await runner.run_debug(
    "Should I vaccinate?"
)


 ### Continue session: debug_session_id

User > Should I vaccinate?
RAG_Vaccine_Informer > Based on the executive briefing provided, there is no information that directly addresses whether you should get vaccinated. The briefing covers developments in Tech, Health, and Finance but does not include any specific recommendations or information about vaccinations.
